## Before you start

This notebook example lets you try out Neptune anonymously, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

## Start a run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass

run = neptune.init_run(
    project="lola-w/hierarchy-tree",  # replace with your own (see instructions below)
    api_token=getpass("eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0ZDgwMjU3Yy0zYWQ1LTQ5ZGMtYjRiZC1kNGE4ZDAzN2JhZGIifQ=="),
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.


### Imports

In [2]:
import torch
from torch import nn
from torch import optim
from torchvision import transforms, datasets
import numpy as np
import networkx
import obonet
import torch
import torch.nn.functional as F
import networkx as nx

### Hyperparameters for training

### Residual - pkk code

In [5]:
onto_path = "http://purl.obolibrary.org/obo/cl/cl-basic.obo"
graph = obonet.read_obo(onto_path)
id_to_name = {id_: data.get('name') for id_, data in graph.nodes(data=True)}
name_to_id = {data['name']: id_  for id_, data in graph.nodes(data=True) if 'name' in data}

def map_celltype(source, target, name_to_id, id_to_name):
    result = []
    for i in networkx.al_simple_paths(graph, name_to_id[source], name_to_id[target]):
        if all(["CL" in j for j in i]):
            result.append([id_to_name[j] for j in i])
    if not result:
        return "others"
    else:
        return result[0][-2]

networkx.is_directed_acyclic_graph(graph)

True

### CustomLoss from Pytorch

In [ ]:
class TreeLoss(nn.Module):
    def __init__(self, graph, lambda_reg):
        super(TreeLoss, self).__init__()
        self.graph = graph
        self.lambda_reg = lambda_reg

    def forward(self, output, target):
        criterion = nn.CrossEntropyLoss()
        ce_loss = criterion(output, target)

        reg_loss = 0.0
        for i in range(len(target)):
            true_label = target[i].item()
            pred_label = output[i].max(0)[1].item()

            if true_label != pred_label:
                common_ancestor = self.find_common_ancestor(true_label, pred_label)
                distance = self.calculate_distance(true_label, common_ancestor) + self.calculate_distance(pred_label, common_ancestor)
                reg_loss += distance
            else:
                # No penalty if the prediction is correct
                reg_loss += 0

        reg_loss /= len(target)
        total_loss = ce_loss + self.lambda_reg * reg_loss
        return total_loss

    def find_common_ancestor(self, type1_id, type2_id):
      # DAG, smaller penalty for predicting a cell type that is an ancestor or descendant of the true cell type
        ancestors1 = set(nx.ancestors(self.graph, type1_id))
        ancestors2 = set(nx.ancestors(self.graph, type2_id))
        common_ancestors = ancestors1.intersection(ancestors2)
        if common_ancestors:
            return min(common_ancestors, key=lambda ancestor: nx.shortest_path_length(self.graph, ancestor, 'root_node_id'))
        return 'root_node_id'  # or another default value

    def calculate_distance(self, cell_type_id, ancestor_id):
      # true vs predicted in a ancestor-descendant relationship
        if ancestor_id == cell_type_id:
            # No penalty for parent-child misclassification
            return 0
        return nx.shortest_path_length(self.graph, ancestor_id, cell_type_id)